In [40]:
from __future__ import print_function
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


In [41]:
data = pd.read_csv('./data.csv')
data.columns = ['Class', 'Data']
print(data[ data['Class'] == 1].size)
print(data[ data['Class'] == 0].size)

#for idx,row in data.iterrows():
#    row[0] = row[0].replace('rt',' ')
    
max_features = 20000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Data'].values)
X = tokenizer.texts_to_sequences(data['Data'].values)
X = pad_sequences(X)

7756
9398


In [42]:
maxlen = 430

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 430, 128)          2560000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 2,659,074
Trainable params: 2,659,074
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
Y = pd.get_dummies(data['Class']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(5746, 430) (5746, 2)
(2831, 430) (2831, 2)


In [44]:
batch_size = 32
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=7,
          validation_data=[X_test, Y_test])

Train on 5746 samples, validate on 2831 samples
Epoch 1/7
5746/5746 [==============================] - 58s 10ms/step - loss: 0.5595 - acc: 0.7027 - val_loss: 0.4082 - val_acc: 0.8135
Epoch 2/7
5746/5746 [==============================] - 60s 10ms/step - loss: 0.2514 - acc: 0.8995 - val_loss: 0.3843 - val_acc: 0.8304
Epoch 3/7
5746/5746 [==============================] - 57s 10ms/step - loss: 0.1078 - acc: 0.9648 - val_loss: 0.4614 - val_acc: 0.8220
Epoch 4/7
5746/5746 [==============================] - 57s 10ms/step - loss: 0.0543 - acc: 0.9827 - val_loss: 0.6997 - val_acc: 0.8276
Epoch 5/7
5746/5746 [==============================] - 57s 10ms/step - loss: 0.0389 - acc: 0.9877 - val_loss: 0.6647 - val_acc: 0.8244
Epoch 6/7
5746/5746 [==============================] - 57s 10ms/step - loss: 0.0214 - acc: 0.9924 - val_loss: 0.6937 - val_acc: 0.8264
Epoch 7/7
5746/5746 [==============================] - 56s 10ms/step - loss: 0.0157 - acc: 0.9957 - val_loss: 0.8722 - val_acc: 0.8138


In [45]:
validation_size = 2500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.5f" % (score))
print("acc: %.5f" % (acc))

score: 1.01092
acc: 0.80060


In [46]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 81.6505706760316 %
neg_acc 81.48420279206466 %


In [52]:
twt = ['দিলিতো দিলি এত বড় বাঁশ দিলি']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=430, dtype='int32', value=0)
#print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

negative


In [38]:
from keras import backend as K
K.clear_session()